In [192]:
!pip install -r requirements.txt

In [193]:
import csv
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [194]:
import datasets
import torchvision.transforms as transforms

tensor_transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.C100Dataset('dataset/data/cifar100_nl.csv', 'dataset/data/cifar100_nl_test.csv')
[trainData, trainLabels, valData, valLabels] = dataset.getDataset()




49999
49999
9999
9999


In [195]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming trainData, trainLabels, valData, and valLabels are numpy arrays
trainData = trainData / 255.0
valData = valData / 255.0

# Convert numpy arrays to torch tensors
trainData = torch.from_numpy(trainData)
trainLabels = torch.from_numpy(trainLabels)
valData = torch.from_numpy(valData)
valLabels = torch.from_numpy(valLabels)

# Convert to float
trainData = trainData.to(torch.float32)
trainLabels = trainLabels.to(torch.float32)
valData = valData.to(torch.float32)
valLabels = valLabels.to(torch.float32)

# Label softening
# label_smoothing = .3
# num_classes = 100

# trainLabels = trainLabels.long()
# valLabels = valLabels.long()

# trainLabels = trainLabels * (1 - label_smoothing) + label_smoothing / num_classes
# valLabels = valLabels * (1 - label_smoothing) + label_smoothing / num_classes

# Create TensorDatasets
trainDataset = TensorDataset(trainData, trainLabels)
valDataset = TensorDataset(valData, valLabels)

# Create DataLoaders
trainLoader = DataLoader(trainDataset, batch_size=64, shuffle=True)
valLoader = DataLoader(valDataset, batch_size=64, shuffle=False)

In [196]:


train_iter = iter(trainLoader)
images, labels = next(train_iter)

images[0].shape


torch.Size([32, 32, 3])

In [197]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
device = get_default_device()

def to_device(entity, device):
    """Move tensor(s) to chosen device"""
    if isinstance(entity, (list,tuple)):
        return [to_device(x, device) for x in entity]
    return entity.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, loader, device):
        self.loader = loader
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for entity in self.loader:
            yield to_device(entity, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.loader)
    
trainLoader = DeviceDataLoader(trainLoader, device)
valLoader = DeviceDataLoader(valLoader, device)

Model Architecture

In [257]:
def divide_and_mix(inputs, targets, alpha):
    batch_size = inputs.size(0)
    index = torch.randperm(batch_size)

    lam = torch.empty(batch_size).uniform_(0, alpha).unsqueeze(1).to(inputs.device)
    mixed_inputs = lam * inputs + (1 - lam) * inputs[index, :]
    expanded_targets = targets.unsqueeze(1).expand(-1, inputs.size(1))
    mixed_targets = lam * expanded_targets + (1 - lam) * expanded_targets[index, :]
    return mixed_inputs, mixed_targets


In [258]:
## ResNet

def conv_block(in_channels, out_channels, pool = False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace = True)]

    if pool:
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool = True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool = True)
        self.conv4 = conv_block(256, 512, pool = True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0,2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, x, targets, alpha):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out

        mixed_inputs, mixed_targets = divide_and_mix(out, targets, alpha)

        out = self.classifier(out)
        return self.classifier(out)


In [259]:
model = ResNet(3, 100)

Model training

In [260]:
def train(model, train_dl, val_dl, epochs, max_lr, loss_func, optim):
    # Initalize the optimizer 
    optimizer = optim(model.parameters(), max_lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs = epochs * len(train_dl), steps_per_epoch = len(train_dl))

    results = []
    lrs = []
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []

        for images, labels in train_dl:
            images = images.permute(0, 3, 1, 2)
            logits = model(images)
            loss = loss_func(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            lrs.append(optimizer.param_groups[0]['lr'])
        epoch_train_loss = torch.stack(train_losses).mean()

        model.eval()
        batch_losses = []
        batch_accs = []
        for images, labels in val_dl:
            with torch.no_grad():
                images = images.permute(0, 3, 1, 2)
                logits = model(images)
            batch_losses.append(loss_func(logits, labels))
            batch_accs.append(accuracy(logits, labels))
        epoch_val_loss = torch.stack(batch_losses).mean().item()
        epoch_val_acc = torch.stack(batch_accs).mean()
        results.append({'avg_validation_loss': epoch_val_loss, 'avg_train_loss': epoch_train_loss, 'avg_validation_accuracy': epoch_val_acc})
    return results

In [261]:
model = to_device(model, device)
epochs = 10
max_lr = 1e-2
loss_func = nn.functional.cross_entropy
optim = torch.optim.Adam

In [262]:
results = train(model, trainLoader, valLoader, epochs, max_lr, loss_func, optim)
for result in results:
    print(result["avg_validation_accuracy"])

RuntimeError: The size of tensor a (64) must match the size of tensor b (4) at non-singleton dimension 2

Evaluation

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in valLoader:
        images, labels = data
        images = images.permute(0,3,1,2)
        outputs = classifier_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the 10000 test images: %d %%' % (100 * correct / total))